# Feature Modeling

This notebook will have an ensemble method composed of lasso linear regression, desicion tree regression, and random forest regression in order to predict Airbnb price listings for rentals.

In [160]:
# Importing libraries
import numpy as np
import pandas as pd
from tabulate import tabulate
import sklearn as skl
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import (
    LinearRegression, Ridge, Lasso, ElasticNet
)
from xgboost import XGBClassifier, XGBRegressor
from sklearn import tree
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, VotingRegressor
from sklearn.metrics import mean_squared_error, r2_score, classification_report

import matplotlib as mpl
import matplotlib.pyplot as plt

# Make this notebook's output stable across runs
random_seed = np.random.seed(100)

In [101]:
# Reading in the CSV file
df = pd.read_excel('../listings.xlsx')


In [102]:
# Drop unnecessary columns
df.drop(labels = ['ID', 'Name', 'Host_ID', 'Host_Name', 'Neighborhood_Group',
       'Neighbourhood', 'Last_Review',
       'Reviews_per_Month', 'Calculated_Host_Listings_Count',
       'Availability_365', 'Number_of_Reviews_LTM', 'Llicense', 'City'], axis=1, inplace=True)

In [103]:
print(df.columns)
# Get Room Dummies
room = pd.get_dummies(df['Room_Type'], prefix='Room Type')
df = pd.concat([df, room], axis=1).drop('Room_Type', axis=1)

# Get State Dummies
state = pd.get_dummies(df['State'], prefix='State')
df = pd.concat([df, state], axis=1).drop('State', axis=1)

# Print the column names
col_data = []
for col in df.columns:
    # assign data
    single_col_data = []
    single_col_data.append(col)
    single_col_data.append(df[col].nunique())
    col_data.append(single_col_data)
 
# create header
head = ["Column", "Count of Unique Values"]
 
# display table
print(tabulate(col_data, headers=head, tablefmt="grid"))

Index(['Latitude', 'Longitude', 'Room_Type', 'Price', 'Minimum_Nights',
       'Number_of_Reviews', 'State', 'MedianIncome', 'MedianAge'],
      dtype='object')
+----------------------------+--------------------------+
| Column                     |   Count of Unique Values |
+============================+==========================+
| Latitude                   |                   143933 |
+----------------------------+--------------------------+
| Longitude                  |                   143607 |
+----------------------------+--------------------------+
| Price                      |                     2528 |
+----------------------------+--------------------------+
| Minimum_Nights             |                      177 |
+----------------------------+--------------------------+
| Number_of_Reviews          |                      750 |
+----------------------------+--------------------------+
| MedianIncome               |                       26 |
+--------------------------

In [104]:
# There are no rows with NaN
NAN = df[df.isna().any(axis=1)]
print(NAN)

Empty DataFrame
Columns: [Latitude, Longitude, Price, Minimum_Nights, Number_of_Reviews, MedianIncome, MedianAge, Room Type_Entire home/apt, Room Type_Hotel room, Room Type_Private room, Room Type_Shared room, State_CA, State_CO, State_District of Columbia, State_FL, State_HI, State_IL, State_LA, State_MA, State_MN, State_NC, State_NJ, State_NV, State_NY, State_OH, State_OR, State_RI, State_TN, State_TX, State_WA]
Index: []

[0 rows x 30 columns]


### Linear Regression

In [130]:
# Split the data into the inputs and output
y = df['Price']
X = df.drop(labels='Price', axis=1)
X2 = df.drop(labels=['Price', 'Latitude', 'Longitude'], axis=1)

In [131]:
# print(X)
# print(y)
print(X2)

        Minimum_Nights  Number_of_Reviews  MedianIncome  MedianAge  \
0                   30                 89         49809       39.0   
1                    1                347         49809       39.0   
2                    1                 67         49809       39.0   
3                    1                296         49809       39.0   
4                   30                 58         49809       39.0   
...                ...                ...           ...        ...   
216308               1                  0         92266       34.0   
216309               1                  0         92266       34.0   
216310               1                  0         92266       34.0   
216311               3                  0         92266       34.0   
216312              91                  0         92266       34.0   

        Room Type_Entire home/apt  Room Type_Hotel room  \
0                               1                     0   
1                               1        

In [106]:
df.dtypes

Latitude                      float64
Longitude                     float64
Price                           int64
Minimum_Nights                  int64
Number_of_Reviews               int64
MedianIncome                    int64
MedianAge                     float64
Room Type_Entire home/apt       uint8
Room Type_Hotel room            uint8
Room Type_Private room          uint8
Room Type_Shared room           uint8
State_CA                        uint8
State_CO                        uint8
State_District of Columbia      uint8
State_FL                        uint8
State_HI                        uint8
State_IL                        uint8
State_LA                        uint8
State_MA                        uint8
State_MN                        uint8
State_NC                        uint8
State_NJ                        uint8
State_NV                        uint8
State_NY                        uint8
State_OH                        uint8
State_OR                        uint8
State_RI    

In [107]:
# Scale data
scaler = StandardScaler()
X_scale = scaler.fit_transform(X)

# Split data into training and test sets
(X_train, X_test,
 y_train, y_test) = train_test_split(X_scale, y, test_size=0.75, random_state=random_seed)

In [132]:
# Scale data for X2
scaler = StandardScaler()
X_scale2 = scaler.fit_transform(X2)

# Split data into training and test sets
(X_train2, X_test2,
 y_train2, y_test2) = train_test_split(X_scale2, y, test_size=0.75, random_state=random_seed)

In [133]:
 #Define functions
lin_reg = LinearRegression()
ridge = Ridge(tol=.001)
lass = Lasso(tol=0.05)
elastic_net = ElasticNet(tol=0.05)

#### Linear Regression

In [134]:
# do Linear Regression
lin_reg_fit = lin_reg.fit(X_train2, y_train2)
y_pred = lin_reg_fit.predict(X_test2)
print("\n Linear Regression: " + str(r2_score(y_test2, y_pred)))


 Linear Regression: 0.03268340413697679


#### Ridge Regression

In [117]:
# do param_grid for Ridge
ridge_search = GridSearchCV(ridge, param_grid={'alpha': [0.01, 0.25, 0.30, 0.40, 0.5, 1, 1.5, 2, 3, 4, 5]}, scoring='r2')
ridge_fit = ridge_search.fit(X_train, y_train)
print("\n RIDGE: The best score across ALL searched params:\n", ridge_search.best_score_)
print("\n RIDGE: The best parameters across ALL searched params:\n", ridge_search.best_params_)


 RIDGE: The best score across ALL searched params:
 0.036597888725485014

 RIDGE: The best parameters across ALL searched params:
 {'alpha': 5}


#### Lasso Regression

In [118]:
# do param_grid for Lasso
lasso_search = GridSearchCV(ridge, param_grid={'alpha' : [0.01, 0.25, 0.50, 0.75, 1, 1.5, 2, 3, 4, 5]}, scoring='r2')
lasso_fit = lasso_search.fit(X_train, y_train)
print("\n LASSO: The best score across ALL searched params:\n", lasso_search.best_score_)
print("\n LASSO: The best parameters across ALL searched params:\n", lasso_search.best_params_)


 LASSO: The best score across ALL searched params:
 0.036597888725485014

 LASSO: The best parameters across ALL searched params:
 {'alpha': 5}


#### Elastic Net Regression

In [119]:
# do param_grid for Elastic
elasticnet_search = GridSearchCV(elastic_net, param_grid={'alpha': [0.01, 0.5, 1, 1.5, 2, 3, 4, 5], 'l1_ratio' : [0.25, 0.30, 0.40, 0.50, 0.75, 1]}, scoring='r2')
elasticnet_fit = elasticnet_search.fit(X_train, y_train)
print("\n ELASTICNET: The best score across ALL searched params:\n", elasticnet_search.best_score_)
print("\n ELASTICNET: The best parameters across ALL searched params:\n", elasticnet_search.best_params_)

C:\Users\carol\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.257e+09, tolerance: 7.565e+08
  model = cd_fast.enet_coordinate_descent(
C:\Users\carol\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.059e+09, tolerance: 7.734e+08
  model = cd_fast.enet_coordinate_descent(
C:\Users\carol\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the sca

KeyboardInterrupt: 

### Decision Tree

In [138]:
tree_reg = DecisionTreeRegressor(max_depth=10, random_state=random_seed)
tree_reg.fit(X_train, y_train)
y_pred = tree_reg.predict(X_train)
print("Whole Dataset Accuracy: ", r2_score(y_train, y_pred))

Whole Dataset Accuracy:  0.5618885610791609


### Random Forest

In [148]:
# Use cross validation to pick the number of trees (estimators)
# and the max depth of the trees
search_parameters = {
    'n_estimators':range(100, 400, 100),
    'max_depth':range(6, 8)
}

rnd_reg = RandomForestRegressor(n_jobs=3, random_state=random_seed) 
grid_reg = GridSearchCV(rnd_reg, search_parameters)
grid_reg.fit(X_train, y_train)

GridSearchCV(estimator=RandomForestRegressor(n_jobs=3),
             param_grid={'max_depth': range(6, 8),
                         'n_estimators': range(100, 400, 100)})

In [149]:
print(f'Accuracy: {grid_reg.best_score_:.3f}')
grid_reg.best_params_

Accuracy: 0.122


{'max_depth': 7, 'n_estimators': 100}

In [151]:
# Get the best classifier from GridSearchCV
best_reg = grid_reg.best_estimator_

# Get accuracy on the test data
y_pred = best_reg.predict(X_train)
print(r2_score(y_train, y_pred))

0.3544509962197674


### XGB Boost Regression

In [153]:
# Create an XGBoost classifier and fit to training data
xgbreg = XGBRegressor(use_label_encoder=False,
                       eval_metric='mlogloss')

xgbreg.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             eval_metric='mlogloss', gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             use_label_encoder=False, validate_parameters=1, verbosity=None)

In [154]:
# Evaluate on test data
y_pred = xgbreg.predict(X_train)
print(r2_score(y_train, y_pred))

0.6925364515575965


## Voting Regressor

In [162]:
voting_reg = VotingRegressor(
    estimators=[('tree', tree_reg), ('ran_forest', rnd_reg),
                ('xbg', xgbreg)])

In [164]:
# Evaluate voting classifier on the training data
voting_reg.fit(X_train, y_train)
y_pred = voting_reg.predict(X_train)
print(r2_score(y_train, y_pred,))

0.7565165415618197


In [167]:
for reg in voting_reg.estimators_:
    reg.fit(X_train, y_train)
    y_pred = reg.predict(X_test)
    print('{:<24} {:.3f}'.format(reg.__class__.__name__,
                                 r2_score(y_test, y_pred)))

DecisionTreeRegressor    -0.124
RandomForestRegressor    0.099
XGBRegressor             0.090


In [171]:
# Fit the voting regressor
voting_reg.fit(X_train, y_train)

idx = 1
instance = X_test[idx, :].to_numpy().reshape(1, -1)

print('{:<40} {:>16}\n'.format('Actual', y_test[idx]))
print('{:<40} {:>16}\n'.format('Voting classifier prediction',
                             voting_reg.predict(instance)[0]))

for clf in voting_reg.estimators_:
    clf.fit(X_train, y_train)
    y_pred = clf.predict(instance)
    print('{:<40} {:>16}'.format(clf.__class__.__name__ + ' prediction',
                                 y_pred[0]))

NameError: name 'instance' is not defined